# **Differentiable Wonderland: Chapter 4**
goal: implement solutions to chapter 4 problems from Alice's Adventures in a Differentiable Wonderland, with particular focus given to...

1) linear models 
2) optimization via gradient descent 
3) train-test splitting and model comparison 

## **Packages and Data**

In [ ]:
import numpy as np
from sklearn import datasets, utils

from sklearn.linear_model import LinearRegression, LogisticRegression 
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier 
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier 

In [323]:
import warnings 
warnings.simplefilter('ignore')

## **Practice Problems**

#### Problem 1

Load a toy dataset: for example, one of those contained in the scikit-learn datasets module.

$\underline{\text{SOLUTION}}$

In [302]:
## dataset for classification - Wisconsin breast cancer 
clf_data = datasets.load_breast_cancer()
print('Successfully loaded classification dataset.')
print(f'- Rows: {clf_data.data.shape[0]:,}')
print(f'- Cols: {clf_data.data.shape[1]:,}')

## dataset for regression - diabetes 
reg_data = datasets.load_diabetes()
print('Successfully loaded regression data.')
print(f'- Rows: {reg_data.data.shape[0]:,}')
print(f'- Cols: {reg_data.data.shape[1]:,}')

Successfully loaded classification dataset.
- Rows: 569
- Cols: 30
Successfully loaded regression data.
- Rows: 442
- Cols: 10


Let's also split our data into training and test sets, which will be useful for later problems. 

In [303]:
def split_data(dataset:utils._bunch.Bunch, train_prop:float, random_seed:int) -> tuple[np.ndarray]: 

    """ given input data and a training ratio, 
        randomly splits data into training and test subsets.  
    """

    ## select training vs. test indices 
    np.random.seed(random_seed) 
    indices = np.arange(dataset.data.shape[0])
    
    n_train = np.ceil(dataset.data.shape[0] * train_prop).astype(int) 
    train_indices = np.random.choice(indices, n_train)
    test_indices = indices[~(np.isin(indices, train_indices))]

    ## create training and test partitions 
    X_train = dataset.data[train_indices] 
    X_test  = dataset.data[test_indices] 

    y_train = dataset.target[train_indices]
    y_test  = dataset.target[test_indices] 

    return (X_train, X_test, y_train, y_test) 

In [304]:
TRAIN_PROP = 0.80
RANDOM_STATE = 15213

## split classification data 
clf_X_train, clf_X_test, clf_y_train, clf_y_test = split_data(clf_data, TRAIN_PROP, RANDOM_STATE)
print('Finished splitting classification data.') 
print(f'- Train Size: {clf_X_train.shape[0]:,}')
print(f'- Test Size:  {clf_X_test.shape[0]:,}')

## split regression data 
reg_X_train, reg_X_test, reg_y_train, reg_y_test = split_data(reg_data, TRAIN_PROP, RANDOM_STATE)
print('Finished splitting regression data.') 
print(f'- Train Size: {reg_X_train.shape[0]:,}')
print(f'- Test Size:  {reg_X_test.shape[0]:,}')

Finished splitting classification data.
- Train Size: 456
- Test Size:  252
Finished splitting regression data.
- Train Size: 354
- Test Size:  197


#### Problem 2

Build a linear model (for regression or classification depending on the dataset). Think about how to make the code as modular as possible: as we will see, you will need at least two functions, one for initializing the parameters of the model and one for computing the model's predictions. 

$\underline{\text{SOLUTION}}$

In [305]:
class LinearModel: 

    """ interface class used for linear models. 
        provides methods for construction and model training. use must implement method for prediction. 
    """

    def __init__(self, X:np.ndarray, y:np.ndarray, bias:bool): 

        ## check input validity 
        try: 
            assert (len(X.shape) == 2)
            assert (len(y.shape) == 1)
        except: 
            raise Exception('X must be 2-dimensional numpy array; y must be 1-dimensional numpy array.')

        ## initialize class members 
        self.bias = bias 
        self.n = X.shape[0] 
        self.p = X.shape[1] + int(bias) 

        self.X = self.calculate_design_matrix(X) 
        self.y = y 

        self.w = np.random.random(self.p) 
        self.grad = np.zeros((self.p,)) 


    def calculate_design_matrix(self, X:np.ndarray): 

        """ translates the input data into a design matrix; only modifies input data if bias is specified.
        """

        design_mat = X
        if (self.bias): 
            design_mat = np.concat([X, np.ones(X.shape[0]).reshape(X.shape[0], 1)], axis=1) 
        
        return design_mat 
    

    def calculate_gradient(self): 

        """ calculates gradient, which is specific to subclass implementation. 
        """

        pass 


    def train(self, n_iter:int, lr:int, momentum:int, verbose:bool=True) -> None: 

        """ optimizes model weights using gradient descent. 
        """

        for _ in range(n_iter): 

            ## calculate gradient over entire dataset 
            grad = self.calculate_gradient() 
            self.grad = grad + momentum * self.grad

            ## perform weight update step 
            self.w = self.w - lr * self.grad 

        print(f'Finished Model Training. Norm of Final Gradient: {grad.T @ grad:.4f}')
            

    def predict(self, X:np.ndarray) -> np.ndarray: 

        """ calculates predictions for input matrix using linear / logistic regression with weight vector. 
        """

        pass 


class LinearRegressor(LinearModel): 

    """ class to create, train, and predict with regards to a linear regression model.
    """

    def __init__(self, X:np.ndarray, y:np.ndarray, bias:bool=True): 
        super(LinearRegressor, self).__init__(X, y, bias)

    
    def calculate_gradient(self): 


        """ calculates gradient for linear regression model.
        """

        return (-2 / self.n) * self.X.T @ (self.y - self.X @ self.w) 


    def predict(self, X:np.ndarray): 

        """ given a group of instances, 
            computes predictions via linear regression + the current weights. 
        """

        ## validate input 
        try: 
            assert (len(X.shape) == 2)
        except: 
            raise Exception('input to predict must be 2-dimensional numpy array.')

        ## convert test input into design matrix 
        X = self.calculate_design_matrix(X) 

        ## calculate predictions
        y_hat = X @ self.w

        return y_hat
    

class BinaryLinearClassifier(LinearModel): 

    """ class to create, train, and predict with regards to a binary logistic regression model.
    """

    def __init__(self, X:np.ndarray, y:np.ndarray, bias:bool=True): 
        super(BinaryLinearClassifier, self).__init__(X, y, bias) 
        self.vectorized_sigmoid = np.vectorize(self.scalar_sigmoid)


    def scalar_sigmoid(self, x:float): 

        """ computes sigmoid for a scalar input. 
        """

        return 1 / (1 + np.exp(-x)) 
    

    def calculate_gradient(self):
        
        """ calculates gradient for binary logistic regression model.
        """

        return (1 / self.n) * self.X.T @ (self.vectorized_sigmoid(self.X @ self.w) - self.y) 
    

    def predict(self, X:np.ndarray): 

        """ given a group of instances, 
            computes predictions via binary logistic regression + the current weights. 
        """
        
        ## validate input 
        try: 
            assert (len(X.shape) == 2)
        except: 
            raise Exception('input to predict must be 2-dimensional numpy array.') 
        
        ## convert test input into design matrix 
        X = self.calculate_design_matrix(X) 

        ## caculate predictions 
        logits = X @ self.w 
        y_hat = self.vectorized_sigmoid(logits) 

        return y_hat

#### Problem 3

Train the model via gradient descent. For now you can compute the gradients maually: try to imagine how you can make also this part modulear, i.e., how do you change the gradient's computation if you want to dynamically add or remove the bias from a model? 

$\underline{\text{SOLUTION}}$

(note that all code for gradient descent is implemented in the previous problem)

Let's start by deriving our gradient for a linear regression model, with a mean squared error loss function:

$$\nabla_w L(w) = \nabla_w \frac{1}{n} \Sigma_i (y_i - w^Tx_i)^2$$
$$\nabla_w L(w) = \nabla_w \frac{1}{n} \Sigma_i (y_i^2 - 2yw^Tx_i + (w^Tx_i)^2)$$
$$\nabla_w L(w) = \frac{1}{n} \Sigma_i (-2yx_i + w^Tx_i^2)$$
$$\nabla_w L(w) = \frac{-2}{n} \Sigma_i x_i(y - w^Tx_i)$$

Next, we'll derive the gradient for a logistic regression model, with a binary cross entropy loss function:

$$\nabla_w L(w) = \nabla_w -\frac{1}{n} \Sigma_i y_i * -\log[\hat{p_i}] + (1 - y_i) * -\log[1 - \hat{p_i}]$$
$$\hat{p} = \sigma(z) = \frac{1}{1 + e^{-z}}, ~~ z = Xw$$
$$\frac{\partial{L}}{\partial{w}} = \frac{\partial{L}}{\partial{y}} \cdot \frac{\partial{y}}{\partial{z}} \cdot \frac{\partial{z}}{\partial{w}}$$
$$\frac{\partial{L}}{\partial{\hat{p}}} = -\frac{y}{\hat{p}} + \frac{1 - y}{1 - \hat{p}} = \frac{\hat{p} - y}{\hat{p}(1 - \hat{p})}, ~~ \frac{\partial{\hat{p}}}{\partial{z}} = \hat{p}(1 - \hat{p}), ~~ \frac{\partial{z}}{\partial{w}} = X$$
$$\frac{\partial{L}}{\partial{w}} = \frac{\hat{p} - y}{\hat{p}(1 - \hat{p})} \cdot \hat{p}(1 - \hat{p}) \cdot X$$
$$\frac{\partial{L}}{\partial{w}} = X(\hat{p} - y)

In [324]:
## train regression model 
reg_mod = LinearRegressor(reg_X_train, reg_y_train, bias=True)
reg_mod.train(n_iter=50_000, lr=0.1, momentum=0.4) 

Finished Model Training. Norm of Final Gradient: 0.0035


In [325]:
## train classification model
clf_mod = BinaryLinearClassifier(clf_X_train, clf_y_train, bias=True) 
clf_mod.train(n_iter=100_000, lr=0.01, momentum=0.2)

Finished Model Training. Norm of Final Gradient: 9071.1961


#### Problem 4

Plot the loss function and the accuracy on an independent test set. If you know some standard machine learning, you can compare the results to other supervised learning models, such as a decision tree or a k-NN model. Use scikit-learn for the other models. 

$\underline{\text{SOLUTION}}$

First, let's create a few functions for our scoring metrics, then apply them to our existing models.

In [308]:
def calc_accuracy(y_true:np.ndarray, y_pred:np.ndarray) -> float: 

    """ given a vector of true labels and a vector of predictions, 
        calculates + returns accuracy score  
    """

    ## validate input 
    try: 
        assert (len(y_true.shape) == 1)
        assert (len(y_pred.shape) == 1) 
    except: 
        raise Exception('input vectors must be 1-dimensional.')
    
    try: 
        assert (y_true.shape[0] == y_pred.shape[0]) 
    except: 
        raise Exception('input vectors must be the same length.') 
    
    ## calculate accuracy 
    n_total = y_true.shape[0] 
    n_correct = y_true[y_true == y_pred].shape[0]

    return n_correct / n_total 


def calc_mean_squared_error(y_true:np.ndarray, y_pred:np.ndarray) -> float: 

    """ given a vector of true labels and a vector of predictions, 
        calculates + returns mean squared error.  
    """

    ## validate input 
    try: 
        assert (len(y_true.shape) == 1)
        assert (len(y_pred.shape) == 1) 
    except: 
        raise Exception('input vectors must be 1-dimensional.')
    
    try: 
        assert (y_true.shape[0] == y_pred.shape[0]) 
    except: 
        raise Exception('input vectors must be the same length.') 
    
    ## calculate mean squared error 
    n_total = y_true.shape[0] 
    squared_resid = y_true - y_pred

    return squared_resid.T @ squared_resid / n_total

In [331]:
## regression 
reg_pred = reg_mod.predict(reg_X_test) 
reg_mse  = calc_mean_squared_error(reg_y_test, reg_pred) 
print(f'Reg Test MSE: {reg_mse:.04f}')

## classification 
clf_pred = clf_mod.predict(clf_X_test) 
clf_acc = calc_accuracy(clf_y_test, clf_pred) 
print(f'Clf Test Acc: {clf_acc:.04f}')

Reg Test MSE: 2940.4807
Clf Test Acc: 0.8810


Now let's compare these scores to models trained via scikit-learn. 

In [332]:
def eval_sklearn_regressor(mod, X_train, X_test, y_train, y_test) -> float: 

    """ given a sklearn regression model and training / test data, 
        trains the model then evaluates predictions made on the test set.  
    """

    mod.fit(X_train, y_train) 
    pred = mod.predict(X_test) 

    score = calc_mean_squared_error(y_test, pred) 
    return score 


def eval_sklearn_classifier(mod, X_train, X_test, y_train, y_test) -> float: 

    """ given a sklearn classification model and training / test data, 
        trains the model then evaluates predictions made on the test set.  
    """

    mod.fit(X_train, y_train) 
    pred = mod.predict(X_test) 

    score = calc_accuracy(y_test, pred) 
    return score

In [345]:
# regression -------------------------------------------

print("Scikit-Learn Regression Models: Test Set Performance") 

## linear regression 
sk_linreg_score = eval_sklearn_regressor(LinearRegression(), reg_X_train, reg_X_test, reg_y_train, reg_y_test)
print(f'- Linear Reg:        {sk_linreg_score:.04f}')

## k-neighbors regression 
sk_knnreg_score = eval_sklearn_regressor(KNeighborsRegressor(), reg_X_train, reg_X_test, reg_y_train, reg_y_test)
print(f'- K-Neighbors:       {sk_knnreg_score:.04f}')

## decision tree regression 
sk_dtreg_score = eval_sklearn_regressor(DecisionTreeRegressor(), reg_X_train, reg_X_test, reg_y_train, reg_y_test)
print(f'- Decision Tree:     {sk_dtreg_score:.04f}')


# classification ---------------------------------------

print("\nScikit-Learn Classification Models: Test Set Performance")

## logistic regression 
sk_logclf_score = eval_sklearn_classifier(LogisticRegression(), reg_X_train, reg_X_test, reg_y_train, reg_y_test)
print(f'- Logistic Reg:  {sk_logclf_score:.04f}')

## k-neighbors classifier 
sk_knnclf_score = eval_sklearn_classifier(KNeighborsClassifier(), reg_X_train, reg_X_test, reg_y_train, reg_y_test)
print(f'- K-Neighbors:   {sk_knnclf_score:.04f}')

## decision tree classifier 
sk_dtclf_score = eval_sklearn_classifier(DecisionTreeClassifier(), reg_X_train, reg_X_test, reg_y_train, reg_y_test)
print(f'- Decision Tree: {sk_dtclf_score:.04f}')

Scikit-Learn Regression Models: Test Set Performance
- Linear Reg:        2975.9868
- K-Neighbors:       3930.4199
- Decision Tree:     6030.0254

Scikit-Learn Classification Models: Test Set Performance
- Logistic Reg:  0.0051
- K-Neighbors:   0.0102
- Decision Tree: 0.0051
